## Blue Ridge Hot Tubs Fixed Charge Problem

Prerequisites: 

In [1]:
!pip install pulp
from pulp import *
import pulp as p

Step 1: Define input parameters

In [2]:
Products = ['Aqua_Spa', 'Hydro_Lux']

Resources = ['Pumps', 'Labor', 'Tubing']

Available_resources = {'Pumps': 200, 'Labor': 1566, 'Tubing': 2880}

Unit_profit = {'Aqua_Spa': 350, 'Hydro_Lux': 300}

Fixed_cost = {'Aqua_Spa': 900, 'Hydro_Lux': 800}

Shared_cost = 1000

Upper_bound = {'Aqua_Spa': 240, 'Hydro_Lux': 261}

Resource_usage = {
    'Aqua_Spa': {'Pumps': 1, 'Labor': 9, 'Tubing': 12},
    'Hydro_Lux': {'Pumps': 1, 'Labor': 6, 'Tubing': 16}
}

Step 2: Define Decision Variables

In [3]:
Units_vars = p.LpVariable.dicts("X", Products, lowBound=0, upBound=None, cat=LpInteger)
Bi_vars = p.LpVariable.dicts("Y", Products, lowBound=0, upBound=1, cat=LpBinary)
Shared_var = p.LpVariable("Z", lowBound=0, upBound=1, cat=LpBinary)


Step 3: Create a variable to contain the problem data

In [4]:
BlueRidge_Problem = p.LpProblem("Blue_Ridge_Hot_Tubs", p.LpMaximize)

Step 4: Define the objective

In [5]:
BlueRidge_Problem += (
    lpSum(Units_vars[i] * Unit_profit[i] for i in Products)
    - lpSum(Bi_vars[i] * Fixed_cost[i] for i in Products)
    - Shared_var * Shared_cost
)

Step 5: Define the constraints

In [6]:
#Resource constraint
for r in Resources:
    BlueRidge_Problem += (
        lpSum(Resource_usage[i][r] * Units_vars[i] for i in Products) <= Available_resources[r]
    ), f"{r}_Constraint"
    
#Linking constraints
for i in Products:
    BlueRidge_Problem += Units_vars[i] <= Upper_bound[i] * Bi_vars[i], f"{i}_Linking_Constraint"

#Shared equipment constraint
BlueRidge_Problem += lpSum(Units_vars[i] for i in Products) <= 200 * Shared_var, "Shared_Equipment_Constraint"


Step 6: Solve the problem and print status

In [7]:
status = BlueRidge_Problem.solve()
print(p.LpStatus[status])

Optimal


Step 7: Print the value of the objective function

In [8]:
print("Total profit = $", p.value(BlueRidge_Problem.objective))

Total profit = $ 63400.0


Step 8: Print the values of the decision variables

In [9]:
for v in BlueRidge_Problem.variables():
    print(v.name, "=", v.varValue)

X_Aqua_Spa = 122.0
X_Hydro_Lux = 78.0
Y_Aqua_Spa = 1.0
Y_Hydro_Lux = 1.0
Z = 1.0
